## Merges going wrong
* There should only be 6 rows for each route (ideally) when its `sched_rt_category` is `schedule_and_vp`
* Dir 1: all day, peak, offpeak
* Dir 0: all day, peak, offpeak.
* This is impacting the graphs for: Timeliness, Frequency of Trips, Average Speed
* December 2024 looks fine but January 2025 is messed up.

In [1]:
import _section1_utils
import _section2_utils
import geopandas as gpd
import merge_data
import numpy as np
import pandas as pd
from segment_speed_utils import gtfs_schedule_wrangling
from shared_utils import rt_dates
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [3]:
analysis_date_list = [rt_dates.y2024_dates[-1]] + rt_dates.y2025_dates

In [4]:
# Test with SF
schd_key = "7cc0cb1871dfd558f11a2885c145d144"

In [5]:
org_name = "City and County of San Francisco"

In [6]:
route_id = "22"

### It seems like `df_sched` is messing everything up because the two values that are supposed to be `peak` and `offpeak` are empty -> Update  `gtfs_funnel/schedule_stats_by_route_direction`

In [7]:
df_sched = merge_data.concatenate_schedule_by_route_direction(analysis_date_list)

In [8]:
df_sched = df_sched.loc[
    (df_sched.schedule_gtfs_dataset_key == schd_key) & (df_sched.route_id == route_id)
]

In [18]:
df_sched.head(2)

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,route_primary_direction,avg_scheduled_service_minutes,avg_stop_miles,n_scheduled_trips,frequency,is_express,is_rapid,is_rail,is_coverage,is_downtown_local,is_local,service_date
9080,7cc0cb1871dfd558f11a2885c145d144,22,0.00,all_day,Southbound,46.87,0.13,186,7.75,0.00,0.00,0.00,0.00,1.00,0.00,2024-12-11
9081,7cc0cb1871dfd558f11a2885c145d144,22,0.00,all_day,Southbound,46.87,4.75,186,7.75,0.00,0.00,0.00,0.00,1.00,0.00,2025-01-15


In [11]:
df_sched[['service_date','time_period','direction_id']].sort_values(by = ['service_date'])

,service_date,time_period,direction_id
9080,2024-12-11,all_day,0.00
9082,2024-12-11,offpeak,0.00
9083,2024-12-11,peak,0.00
9086,2024-12-11,all_day,1.00
9088,2024-12-11,offpeak,1.00
9089,2024-12-11,peak,1.00
9081,2025-01-15,all_day,0.00
9084,2025-01-15,None,0.00
9085,2025-01-15,None,0.00
9087,2025-01-15,all_day,1.00


In [12]:
df_avg_speeds = merge_data.concatenate_speeds_by_route_direction(
        analysis_date_list
    )

In [13]:
df_avg_speeds.sample()

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,speed_mph,service_date
7568,7cc0cb1871dfd558f11a2885c145d144,F,0.00,offpeak,6.09,2025-01-15


In [15]:
df_avg_speeds = df_avg_speeds.loc[
    (df_avg_speeds.schedule_gtfs_dataset_key == schd_key) & (df_avg_speeds.route_id == route_id)
]

In [17]:
df_avg_speeds.sort_values(by = ['service_date'])

,schedule_gtfs_dataset_key,route_id,direction_id,time_period,speed_mph,service_date
7076,7cc0cb1871dfd558f11a2885c145d144,22,0.00,all_day,5.36,2024-12-11
7078,7cc0cb1871dfd558f11a2885c145d144,22,0.00,offpeak,5.34,2024-12-11
7080,7cc0cb1871dfd558f11a2885c145d144,22,0.00,peak,5.37,2024-12-11
7082,7cc0cb1871dfd558f11a2885c145d144,22,1.00,all_day,6.05,2024-12-11
7084,7cc0cb1871dfd558f11a2885c145d144,22,1.00,offpeak,6.28,2024-12-11
7086,7cc0cb1871dfd558f11a2885c145d144,22,1.00,peak,5.80,2024-12-11
7077,7cc0cb1871dfd558f11a2885c145d144,22,0.00,all_day,5.78,2025-01-15
7079,7cc0cb1871dfd558f11a2885c145d144,22,0.00,offpeak,5.92,2025-01-15
7081,7cc0cb1871dfd558f11a2885c145d144,22,0.00,peak,5.63,2025-01-15
7083,7cc0cb1871dfd558f11a2885c145d144,22,1.00,all_day,6.38,2025-01-15
